<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EB%B6%84%EB%A5%982.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Lightgbm

* 데이터 '와인품질분류.zip'입니다.

1. 필요한 라이브러리를 가져옵니다.

In [100]:
import pandas as pd
import lightgbm as lgbm

2. 데이터를 불러옵니다.

In [101]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-11-14 10:16:54--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-11-14 10:16:54--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 173.194.214.100, 173.194.214.113, 173.194.214.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.214.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n0s0ate7s19npvc1rk9vggmdon0jj9dt/1668420975000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download&uuid=e196ebc1-e3f1-439d-bfca-188a1f6a454a [following]
--2022-11-14 10:16:54--  https://doc-10-10-doc

In [102]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
smpl_sub = pd.read_csv('data/sample_submission.csv')

In [103]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [104]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


3. 변수를 변환합니다. 

In [105]:
# type에는 white와 red 두 종류가 있습니다.
# 각각 0,1로 변환합니다.
# 아래의 방법을 사용하면 0,1순서가 바뀜
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [106]:
# 방법2
# from sklearn.preprocessing import LabelEncoder

# enc = LabelEncoder()
# enc.fit(train['type'])
# train['type'] = enc.transform(train['type'])
# test['type'] = enc.transform(test['type'])

In [108]:
# 오류남. 
# train['type'] = train['type'].map({'white':0, 'red':1}).astype(int)
# test['type'] = test['type'].map({'white':0, 'red':1}).astype(int)

In [109]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0,1
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1,0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0,1
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0,1
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0,1


In [110]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0,1
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,1,0
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0,1
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0,1
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0,1


In [111]:
# 모델에 입력하기 전 데이터를 정형합니다.
# train_x : train의 'index', 'quality' 피처를 제거하고 저장합니다.
# train_y : train의 'quality'를 저장합니다.
# test_x : test의 'index'를 제거하고 저장합니다.

train_x = train.drop(['index','quality'], axis=1)
train_y = train.loc[:, train.columns == 'quality']
test_x = test.loc[:, test.columns != 'index']

In [112]:
train_x.shape, train_y.shape, test_x.shape

((5497, 13), (5497, 1), (1000, 13))

In [113]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0,1
1,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1,0
2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0,1
3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0,1
4,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0,1


In [114]:
train_y.head()

,quality
0,5
1,5
2,5
3,6
4,6


In [115]:
test_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0,1
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,1,0
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0,1
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0,1
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0,1


4. LightGBM 모델을 생성 및 훈련합니다.

In [116]:
# LightGBM의 파이썬 패키지인 lightgbm 에서 LGBMClassifier 임포트
from lightgbm import LGBMClassifier

import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# n_estimators는 400 설정
lgbm_wrapper = LGBMClassifier(n_estimators=400)

# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능하지만, 굳이 하지 않겠음. 
evals = [(train_x, train_y)]
lgbm_wrapper.fit(train_x, train_y, eval_metric="logloss", eval_set=evals, verbose=True)

[1]	training's multi_logloss: 1.21288	training's multi_logloss: 1.21288
[2]	training's multi_logloss: 1.16896	training's multi_logloss: 1.16896
[3]	training's multi_logloss: 1.13131	training's multi_logloss: 1.13131
[4]	training's multi_logloss: 1.09957	training's multi_logloss: 1.09957
[5]	training's multi_logloss: 1.07092	training's multi_logloss: 1.07092
[6]	training's multi_logloss: 1.04486	training's multi_logloss: 1.04486
[7]	training's multi_logloss: 1.02055	training's multi_logloss: 1.02055
[8]	training's multi_logloss: 0.998871	training's multi_logloss: 0.998871
[9]	training's multi_logloss: 0.979262	training's multi_logloss: 0.979262
[10]	training's multi_logloss: 0.961149	training's multi_logloss: 0.961149
[11]	training's multi_logloss: 0.943494	training's multi_logloss: 0.943494
[12]	training's multi_logloss: 0.927153	training's multi_logloss: 0.927153
[13]	training's multi_logloss: 0.911194	training's multi_logloss: 0.911194
[14]	training's multi_logloss: 0.896552	training

LGBMClassifier(n_estimators=400)

5. 생성한 모델로 예측 데이터를 생성합니다.

In [117]:
preds = lgbm_wrapper.predict(test_x)
pred_proba = lgbm_wrapper.predict_proba(test_x)[:, 1]

In [118]:
preds[:5] # quality 예측 값들의 나열

array([5, 6, 6, 5, 6])

6. 제출파일을 생성합니다.

In [119]:
smpl_sub.head()

,index,quality
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [120]:
#서브미션 파일의 quality 변수에 예측값을 대입합니다.
smpl_sub['quality'] = preds 

In [121]:
smpl_sub.head()

,index,quality
0,0,5
1,1,6
2,2,6
3,3,5
4,4,6


In [122]:
# csv 파일로 저장합니다.
smpl_sub.to_csv('submission.csv', index=False)

# 2. XGBoost

In [123]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-11-14 10:17:01--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-11-14 10:17:01--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 173.194.214.100, 173.194.214.113, 173.194.214.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.214.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n0s0ate7s19npvc1rk9vggmdon0jj9dt/1668420975000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download&uuid=02dccaff-7516-4583-af5c-33047d982de0 [following]
--2022-11-14 10:17:02--  https://doc-10-10-doc

In [124]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [125]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [126]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


In [127]:
# 원핫 인코딩 (pd.get_dummies())
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [128]:
# 모델 정의
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)

In [129]:
y_train.head()

,quality
0,5
1,5
2,5
3,6
4,6


In [130]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X_train = train.drop(['quality'], axis=1)
y_train = train.loc[:, train.columns == 'quality']

# dtrain = xgb.DMatrix(data=X_train, label=y_train)
# dtest = xgb.DMatrix(data=test, label=y_test)


# fit 메소드를 이용해 모델 학습
# train 데이터 세트는 'train', evaluation(test) 데이터 세트는 eval로 명기함.
xgb_wrapper.fit(X_train, y_train)

XGBClassifier(n_estimators=400, objective='multi:softprob')

In [131]:
# predict 메소드와 test_one 데이터를 이용해 품질 예측
w_preds = xgb_wrapper.predict(test)

w_preds[:5]

array([6, 5, 5, 6, 6])

In [132]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기
smpl_sub2 = pd.read_csv('data/sample_submission.csv')
smpl_sub2.head()

,index,quality
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [133]:
smpl_sub2['quality'] = preds 
smpl_sub2.head()

,index,quality
0,0,5
1,1,6
2,2,6
3,3,5
4,4,6


In [134]:
# 정답파일 내보내기
smpl_sub2.to_csv('xgb_pred.csv',index = False)